# Experimental

Messing around with curve fits

Source data: 

* https://www.census.gov/data/tables/time-series/demo/income-poverty/cps-hinc/hinc-01.html
* https://www2.census.gov/programs-surveys/cps/tables/hinc-01/2024/hinc01_1.xlsx

In [6]:
from scipy.stats import gamma
import numpy as np
import pandas as pd
import scipy.stats as stats
import ace_tools_open as tools


In [7]:
# Original data

bin_descriptions = [
    "<5k", "5-10k", "10k-15k", "15k-20k", "20k-25k", "25k-30k", "30k-35k", "35k-40k",
    "40k-45k", "45k-50k", "50k-55k", "55k-60k", "60k-65k", "65k-70k", "70k-75k",
    "75k-80k", "80k-85k", "85k-90k", "90k-95k", "95k-100k", "100k-105k", "105k-110k",
    "110k-115k", "115k-120k", "120k-125k", "125k-130k", "130k-135k", "135k-140k",
    "140k-145k", "145k-150k", "150k-155k", "155k-160k", "160k-165k", "165k-170k",
    "170k-175k", "175k-180k", "180k-185k", "185k-190k", "190k-195k", "195k-200k"
]

midpoints = [
    2500, 7500, 12500, 17500, 22500, 27500, 32500, 37500, 42500, 47500, 52500, 57500,
    62500, 67500, 72500, 77500, 82500, 87500, 92500, 97500, 102500, 107500, 112500,
    117500, 122500, 127500, 132500, 137500, 142500, 147500, 152500, 157500, 162500,
    167500, 172500, 177500, 182500, 187500, 192500, 197500
]


# # Input data excluding the >200k bin
# data = {
#     "Bin": [
#         "<5k", "5-10k", "10k-15k", "15k-20k", "20k-25k", "25k-30k", "30k-35k", "35k-40k",
#         "40k-45k", "45k-50k", "50k-55k", "55k-60k", "60k-65k", "65k-70k", "70k-75k",
#         "75k-80k", "80k-85k", "85k-90k", "90k-95k", "95k-100k", "100k-105k", "105k-110k",
#         "110k-115k", "115k-120k", "120k-125k", "125k-130k", "130k-135k", "135k-140k",
#         "140k-145k", "145k-150k", "150k-155k", "155k-160k", "160k-165k", "165k-170k",
#         "170k-175k", "175k-180k", "180k-185k", "185k-190k", "190k-195k", "195k-200k"
#     ],
#     "Pct": [
#         3.08, 1.35, 3.00, 3.25, 3.44, 3.48, 3.47, 3.50, 3.33, 3.42, 3.57, 3.10,
#         3.41, 2.85, 2.76, 2.59, 2.65, 2.38, 2.47, 2.04, 2.47, 1.91, 1.91, 1.82,
#         1.90, 1.52, 1.56, 1.36, 1.40, 1.15, 1.57, 1.06, 1.11, 0.95, 0.92, 0.82,
#         0.98, 0.74, 0.71, 0.58
#     ]
# }

actual_percentages = [
    0.0308, 0.0135, 0.0300, 0.0325, 0.0344, 0.0348, 0.0347, 0.0350, 0.0333, 0.0342,
    0.0357, 0.0310, 0.0341, 0.0285, 0.0276, 0.0259, 0.0265, 0.0238, 0.0247, 0.0204,
    0.0247, 0.0191, 0.0191, 0.0182, 0.0190, 0.0152, 0.0156, 0.0136, 0.0140, 0.0115,
    0.0157, 0.0106, 0.0111, 0.0095, 0.0092, 0.0082, 0.0098, 0.0074, 0.0071, 0.0058
]

# Convert percentages to probabilities
probabilities = actual_percentages

In [8]:
# Expand the data using midpoints and probabilities for fitting
expanded_data = []
for midpoint, prob in zip(midpoints, actual_percentages):
    count = int(prob * 1000)  # Scale factor for expansion
    expanded_data.extend([midpoint] * count)

In [ ]:
import pandas as pd
from scipy.stats import lognorm, norm, gamma, expon, kstest

# List of distributions to test
distributions = {
    "Lognormal": lognorm,
    "Normal": norm,
    "Gamma": gamma,
    "Exponential": expon
}

# Fit each distribution and perform the Kolmogorov-Smirnov (KS) test
fit_results = {}

for dist_name, dist in distributions.items():
    # Fit the distribution to the data
    if dist_name in ["Gamma", "Exponential"]:  # These require a non-negative location
        params = dist.fit(expanded_data, floc=0)
    else:
        params = dist.fit(expanded_data)
    
    # Perform KS test
    cdf_func = lambda x: dist.cdf(x, *params)
    ks_stat, ks_p_value = kstest(expanded_data, cdf_func)
    
    # Store results
    fit_results[dist_name] = {
        "Parameters": params,
        "KS Statistic": ks_stat,
        "P-Value": ks_p_value
    }

# Compile results into a DataFrame
fit_comparison_df = pd.DataFrame(fit_results).T

# Display the comparison of fits
print(fit_comparison_df)


In [ ]:

# Extend midpoints and bin descriptions to 500k
new_bins_count = 61  # Additional bins beyond 200k
new_midpoints = [midpoints[-1] + 5000 * i for i in range(1, new_bins_count + 1)]
new_bin_descriptions = [
    f"{(200000 + 5000 * i)//1000}k-{(200000 + 5000 * (i + 1))//1000}k" for i in range(new_bins_count - 1)
]
new_bin_descriptions.append(">500k")  # Final bin description for >500k

#new_midpoints.append(500000)

# Combine with original midpoints and bins
extended_midpoints = midpoints + new_midpoints
extended_bins = bin_descriptions + new_bin_descriptions

# Extend actual percentages with NaN for bins beyond 200k
extended_actual_percentages = actual_percentages + [None] * (len(extended_midpoints) - len(midpoints))

# Assign dist parameters
gamma_a, gamma_loc, gamma_scale = fit_comparison_df.loc["Gamma", "Parameters"]
lognorm_shape, lognorm_loc, lognorm_scale = fit_comparison_df.loc["Lognormal", "Parameters"]

# Calculate gamma fit percentages for the extended range
extended_gamma_fit_percentages = []
extended_lognorm_fit_percentages = []

for i in range(len(extended_midpoints)):
    lower_bound = extended_midpoints[i] - 2500 if i > 0 else 0  # Adjust for bin width
    upper_bound = extended_midpoints[i] + 2500
    
    # Gamma fit
    gamma_fit_percentage = gamma.cdf(upper_bound, gamma_a, loc=gamma_loc, scale=gamma_scale) - \
                           gamma.cdf(lower_bound, gamma_a, loc=gamma_loc, scale=gamma_scale)
    extended_gamma_fit_percentages.append(gamma_fit_percentage)
    
    # Log-normal fit
    lognorm_fit_percentage = lognorm.cdf(upper_bound, lognorm_shape, loc=lognorm_loc, scale=lognorm_scale) - \
                             lognorm.cdf(lower_bound, lognorm_shape, loc=lognorm_loc, scale=lognorm_scale)
    extended_lognorm_fit_percentages.append(lognorm_fit_percentage)

# Create the extended DataFrame
extended_comparison_df = pd.DataFrame({
    "Original Bin": extended_bins,
    "Bin Midpoints": extended_midpoints,
    "Actual Percentages": extended_actual_percentages,
    "Gamma Fit Percentages": extended_gamma_fit_percentages,
    "Log-Normal Fit Percentages": extended_lognorm_fit_percentages
})

# Display the results
print(extended_comparison_df)

In [ ]:
import matplotlib.pyplot as plt

# Extract data for plotting
midpoints = extended_comparison_df["Bin Midpoints"]
actual_percentages = extended_comparison_df["Actual Percentages"]
gamma_fit_percentages = extended_comparison_df["Gamma Fit Percentages"]
lognorm_fit_percentages = extended_comparison_df["Log-Normal Fit Percentages"]

# Plot actual percentages as bars
plt.figure(figsize=(14, 8))
plt.bar(midpoints, actual_percentages, width=4000, alpha=0.6, label="Actual Percentages", color="skyblue")

# Plot gamma fit percentages as a line
plt.plot(midpoints, gamma_fit_percentages, label="Gamma Fit Percentages", color="orange", linewidth=2)

# Plot log-normal fit percentages as a line
plt.plot(midpoints, lognorm_fit_percentages, label="Log-Normal Fit Percentages", color="green", linewidth=2)

# Formatting the plot
plt.title("Comparison of Actual vs Gamma and Log-Normal Fit Percentages", fontsize=16)
plt.xlabel("Income Bin Midpoints ($)", fontsize=14)
plt.ylabel("Percentage of Households", fontsize=14)
plt.legend(fontsize=12)
plt.grid(alpha=0.3)
plt.xticks(ticks=midpoints[::5], labels=[f"{int(m/1000)}k" for m in midpoints[::5]], rotation=45)
plt.tight_layout()
plt.show()


# Geometric?

In [ ]:
from scipy.optimize import curve_fit

# Define geometric decay model
def geometric_decay(x, y0, r):
    return y0 * r**x

# Extract data starting from around 50,000
start_index = np.where(filtered_midpoints >= 50000)[0][0]
geom_midpoints = filtered_midpoints[start_index:]
geom_probabilities = filtered_probabilities[start_index:]

# Convert midpoints to indices for the decay model
indices = np.arange(len(geom_midpoints))

# Fit the geometric decay model
popt, _ = curve_fit(geometric_decay, indices, geom_probabilities, p0=[geom_probabilities[0], 0.9])

# Generate the fitted curve
fitted_geom_curve = geometric_decay(indices, *popt)

# Extend indices to project the geometric curve to $500,000
extended_indices = np.arange(len(geom_midpoints) + 100)
extended_geom_curve = geometric_decay(extended_indices, *popt)
extended_midpoints = np.concatenate([geom_midpoints, 
                                      geom_midpoints[-1] + np.arange(1, len(extended_indices) - len(geom_midpoints) + 1) * 5000])

# Plot the full dataset and extended geometric decay curve
plt.figure(figsize=(12, 6))
plt.bar(filtered_midpoints, filtered_probabilities, width=3000, alpha=0.6, label="Original Data")
plt.plot(extended_midpoints, extended_geom_curve, label="Geometric Decay Fit (Extended)", linestyle="--", linewidth=2)
plt.title("Full Dataset with Extended Geometric Decay Curve")
plt.xlabel("Income ($)")
plt.ylabel("Probability")
#plt.axvline(x=500000, color="red", linestyle=":", label="Projection to $500,000")
plt.legend()
plt.grid()
plt.show()

In [ ]:
extended_geom_curve


In [ ]:
# Align the length of extended_geom_curve to match extended_midpoints
geom_fit_full_padded = np.concatenate([[None] * start_index, extended_geom_curve])

# Adjust all columns to the same length
actual_data_padded = np.concatenate([filtered_probabilities, [None] * (len(extended_midpoints) - len(filtered_probabilities))])
geom_fit_full_padded = geom_fit_full_padded[:len(extended_midpoints)]

# Create the updated DataFrame
results_df_updated = pd.DataFrame({
    "Midpoints": extended_midpoints,
    "Actual Data (Probabilities)": actual_data_padded,
    "Geometric Curve Fit (>50k)": geom_fit_full_padded
})

#results_df_updated['cum_geo'] = results_df_updated['Geometric Curve Fit (>50k)'].cumsum()

tools.display_dataframe_to_user(name="Updated Geometric Decay Fit Results", dataframe=results_df_updated)


In [ ]:
results_df_updated.loc[results_df_updated['Midpoints'] > 500000, 'Geometric Curve Fit (>50k)'].sum()